# 猫狗大战 毕业项目——Fine-tuning Xception

## 开始
导入一切并我们设置所使用的GPU。
- dev0: GTX1070Ti

In [1]:
%matplotlib inline
#import utilities
import os
import shutil
import numpy as np
import random
from tqdm import tqdm  
from time import time
from PIL import Image
import h5py
import pandas as pd
from helper import *

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.optimizers import *
from keras.utils import *
from keras import backend as K

#如果系统上有多块GPU，“0”可以替换成其它GPU的编号
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/pengjun/.conda/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 数据文件处理
训练数据包括12500张猫的图片和12500张狗的图片。我们为数据文件建立symbol link并划分为训练集和验证集，所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

In [2]:
#为数据连理symbol-link
train_data_dir, valid_data_dir, test_data_dir = prepare_data_file()

100%|██████████| 12500/12500 [00:00<00:00, 189498.72it/s]


## 基准模型
作为迁移学习的基础，这里我们使用Xception为基准模型：
- [Xception](https://arxiv.org/abs/1610.02357)

在导出预训练模型特征时，我们所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

我们首先冻结所有Xception的权重参数，只训练全链接层。我们在融合模型中已经导出了所有训练数据和测试数据在Xception上的特征，基于这些特征，我们训练猫狗问题的网络。

In [3]:
#导入训练数据和测试数据
X_train, Y_train, X_test = load_feature_data("feature_xception.h5")

In [4]:
#构造模型并显示所有网络层的名称
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [5]:
#训练模型并导出权重参数
filepath="xception-tune0-best_weight.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True,
         callbacks=callbacks_list)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 1s 61us/step - loss: 0.1565 - acc: 0.9689 - val_loss: 0.0468 - val_acc: 0.9932

Epoch 00001: val_loss improved from inf to 0.04683, saving model to xception-tune0-best_weight.h5
Epoch 2/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.0407 - acc: 0.9911 - val_loss: 0.0246 - val_acc: 0.9952

Epoch 00002: val_loss improved from 0.04683 to 0.02456, saving model to xception-tune0-best_weight.h5
Epoch 3/20
20000/20000 [==============================] - 1s 26us/step - loss: 0.0287 - acc: 0.9922 - val_loss: 0.0194 - val_acc: 0.9950

Epoch 00003: val_loss improved from 0.02456 to 0.01939, saving model to xception-tune0-best_weight.h5
Epoch 4/20
20000/20000 [==============================] - 1s 27us/step - loss: 0.0252 - acc: 0.9926 - val_loss: 0.0168 - val_acc: 0.9960

Epoch 00004: val_loss improved from 0.01939 to 0.01680, saving model to xception-tune0-best_weigh

In [6]:
#在测试集上进行预测并导出预测值
predict_on_model(test_data_dir, X_test, model, "pred-xception-tune0.csv")

12500/12500 [==============================] - 0s 29us/step
Found 12500 images belonging to 1 classes.


/home/pengjun/DLND/Cat_vs_Dog/helper.py:130: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(index-1, 'label', y_test[i])


> 这一模型在Kaggle上的得分为0.04091，非常高的一个分数。下一步我们将开始Fine-tuning基于Xception的猫狗分类模型。

## Fine Tuning
我们将放开Xception中的一些单元的权值，让它们是可学习的，以此训练我们的猫狗分类网络。

### Fine Tune freeze
这里，我们首先在冻结全部权重情况下对全链接层进行训练。我们引入数据增强以获得更为泛化的数据集。

In [3]:
#构造模型
x_input = Input((299, 299, 3))
x_input = Lambda(xception.preprocess_input)(x_input)

base_model = Xception(input_tensor=x_input, weights='imagenet', include_top=False, pooling = 'avg')

for layer in base_model.layers:
    layer.trainable = False
    
x = Dropout(0.5)(base_model.output)
x = Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.001))(x)
model = Model(base_model.input, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

gen = ImageDataGenerator(rotation_range=90,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)
val_gen = ImageDataGenerator()
train_generator = gen.flow_from_directory(train_data_dir, (299, 299), shuffle=True, 
                                          batch_size=64,class_mode='binary')
valid_generator = val_gen.flow_from_directory(valid_data_dir, (299, 299), shuffle=True, 
                                          batch_size=32,class_mode='binary')

for i in range(len(model.layers)):
    print(i,model.layers[i].name)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
0 input_1
1 lambda_1
2 block1_conv1
3 block1_conv1_bn
4 block1_conv1_act
5 block1_conv2
6 block1_conv2_bn
7 block1_conv2_act
8 block2_sepconv1
9 block2_sepconv1_bn
10 block2_sepconv2_act
11 block2_sepconv2
12 block2_sepconv2_bn
13 conv2d_1
14 block2_pool
15 batch_normalization_1
16 add_1
17 block3_sepconv1_act
18 block3_sepconv1
19 block3_sepconv1_bn
20 block3_sepconv2_act
21 block3_sepconv2
22 block3_sepconv2_bn
23 conv2d_2
24 block3_pool
25 batch_normalization_2
26 add_2
27 block4_sepconv1_act
28 block4_sepconv1
29 block4_sepconv1_bn
30 block4_sepconv2_act
31 block4_sepconv2
32 block4_sepconv2_bn
33 conv2d_3
34 block4_pool
35 batch_normalization_3
36 add_3
37 block5_sepconv1_act
38 block5_sepconv1
39 block5_sepconv1_bn
40 block5_sepconv2_act
41 block5_sepconv2
42 block5_sepconv2_bn
43 block5_sepconv3_act
44 block5_sepconv3
45 block5_sepconv3_bn
46 add_4
47 block6_sepconv1_act
48 block6_sepconv1
49 bl

In [8]:
#训练模型并保存在验证集上损失函数最小的权重
filepath="xception-best_weight_freeze.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=625,
        epochs=5,
        validation_data=valid_generator,
        validation_steps=150,
        callbacks = callbacks_list)

Epoch 1/5
625/625 [==============================] - 510s 815ms/step - loss: 0.1936 - acc: 0.9349 - val_loss: 0.0794 - val_acc: 0.9846

Epoch 00001: val_loss improved from inf to 0.07936, saving model to xception-best_weight_freeze.h5
Epoch 2/5
625/625 [==============================] - 513s 821ms/step - loss: 0.1408 - acc: 0.9500 - val_loss: 0.0816 - val_acc: 0.9825

Epoch 00002: val_loss did not improve
Epoch 3/5
625/625 [==============================] - 511s 818ms/step - loss: 0.1366 - acc: 0.9517 - val_loss: 0.0618 - val_acc: 0.9892

Epoch 00003: val_loss improved from 0.07936 to 0.06180, saving model to xception-best_weight_freeze.h5
Epoch 4/5
625/625 [==============================] - 513s 821ms/step - loss: 0.1322 - acc: 0.9536 - val_loss: 0.0849 - val_acc: 0.9819

Epoch 00004: val_loss did not improve
Epoch 5/5
625/625 [==============================] - 511s 817ms/step - loss: 0.1360 - acc: 0.9519 - val_loss: 0.0788 - val_acc: 0.9838

Epoch 00005: val_loss did not improve


In [9]:
predict_on_xception(12500, 299, 299, test_data_dir, model, "xception-best_weight_freeze.h5", "pred-xception-freeze.csv")

100%|██████████| 12500/12500 [00:43<00:00, 287.54it/s]


12500/12500 [==============================] - 127s 10ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:166: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 237378.32it/s]


> 这个模型在kaggle的得分是0.07127。

### Fine-tuning 1
开放97层以上的权重优化

In [10]:
for layer in model.layers:
    layer.trainable = False
    
for layer in model.layers[97:]:
    layer.trainable = True
    
model.load_weights('xception-best_weight_freeze.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
#训练模型并保存在验证集上损失函数最小的权重
filepath="xception-best_weight_fine_tuning-1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=1250,
        epochs=5,
        validation_data=valid_generator,
        validation_steps=150,
        callbacks = callbacks_list)

Epoch 1/5
1250/1250 [==============================] - 999s 799ms/step - loss: 0.0547 - acc: 0.9819 - val_loss: 0.0205 - val_acc: 0.9944

Epoch 00001: val_loss improved from inf to 0.02055, saving model to xception-best_weight_fine_tuning-1.h5
Epoch 2/5
1250/1250 [==============================] - 996s 797ms/step - loss: 0.0253 - acc: 0.9925 - val_loss: 0.0187 - val_acc: 0.9950

Epoch 00002: val_loss improved from 0.02055 to 0.01867, saving model to xception-best_weight_fine_tuning-1.h5
Epoch 3/5
1250/1250 [==============================] - 997s 797ms/step - loss: 0.0170 - acc: 0.9950 - val_loss: 0.0162 - val_acc: 0.9954

Epoch 00003: val_loss improved from 0.01867 to 0.01621, saving model to xception-best_weight_fine_tuning-1.h5
Epoch 4/5
1250/1250 [==============================] - 997s 798ms/step - loss: 0.0140 - acc: 0.9961 - val_loss: 0.0196 - val_acc: 0.9944

Epoch 00004: val_loss did not improve
Epoch 5/5
1250/1250 [==============================] - 1004s 803ms/step - loss: 0.01

In [ ]:
predict_on_xception(12500, 299, 299, test_data_dir, model, "xception-best_weight_fine_tuning-1.h5", "pred-xception-fine_tuning-1.csv")

100%|██████████| 12500/12500 [00:43<00:00, 288.74it/s]


 6496/12500 [==============>...............] - ETA: 1:02

> 这一轮Tuning的最终得分为：0.03600。

### Fine-tuning 2
开放107层以上的权重优化

In [ ]:
for layer in model.layers:
    layer.trainable = False
    
for layer in model.layers[107:]:
    layer.trainable = True
    
model.load_weights('xception-best_weight_fine_tuning-1.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
filepath="xception-best_weight_fine_tuning-2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=1250,
        epochs=5,
        validation_data=valid_generator,
        validation_steps=150,
        callbacks = callbacks_list)

1250/1250 [==============================] - 991s 793ms/step - loss: 0.0123 - acc: 0.9964 - val_loss: 0.0228 - val_acc: 0.9938

Epoch 00001: val_loss improved from inf to 0.02279, saving model to xception-best_weight_fine_tuning-2.h5
Epoch 2/5
1250/1250 [==============================] - 992s 794ms/step - loss: 0.0102 - acc: 0.9971 - val_loss: 0.0263 - val_acc: 0.9935

Epoch 00002: val_loss did not improve
Epoch 3/5
1250/1250 [==============================] - 988s 790ms/step - loss: 0.0091 - acc: 0.9977 - val_loss: 0.0259 - val_acc: 0.9923

Epoch 00003: val_loss did not improve
Epoch 4/5
1250/1250 [==============================] - 989s 791ms/step - loss: 0.0080 - acc: 0.9980 - val_loss: 0.0166 - val_acc: 0.9960

Epoch 00004: val_loss improved from 0.02279 to 0.01660, saving model to xception-best_weight_fine_tuning-2.h5
Epoch 5/5
1250/1250 [==============================] - 988s 791ms/step - loss: 0.0070 - acc: 0.9984 - val_loss: 0.0243 - val_acc: 0.9938

Epoch 00005: val_loss did no

In [15]:
predict_on_xception(12500, 299, 299, test_data_dir, model, "xception-best_weight_fine_tuning-2.h5", "pred-xception-fine_tuning-2.csv")

100%|██████████| 12500/12500 [00:43<00:00, 288.82it/s]


12500/12500 [==============================] - 129s 10ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:166: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 242560.85it/s]


> 这一轮Tuning的最终得分为：0.03780。

### Fine-tuning 3
开放87层以上的权重优化。

In [9]:
for layer in model.layers:
    layer.trainable = False
    
for layer in model.layers[87:]:
    layer.trainable = True
    
model.load_weights('xception-best_weight_fine_tuning-1.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
filepath="xception-best_weight_fine_tuning-3.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=1250,
        epochs=5,
        validation_data=valid_generator,
        validation_steps=150,
        callbacks = callbacks_list)

Epoch 1/5
1250/1250 [==============================] - 1085s 868ms/step - loss: 0.0153 - acc: 0.9957 - val_loss: 0.0175 - val_acc: 0.9946

Epoch 00001: val_loss improved from inf to 0.01750, saving model to xception-best_weight_fine_tuning-3.h5
Epoch 2/5
1250/1250 [==============================] - 1084s 867ms/step - loss: 0.0110 - acc: 0.9970 - val_loss: 0.0225 - val_acc: 0.9944

Epoch 00002: val_loss did not improve
Epoch 3/5
1250/1250 [==============================] - 1084s 867ms/step - loss: 0.0093 - acc: 0.9977 - val_loss: 0.0207 - val_acc: 0.9944

Epoch 00003: val_loss did not improve
Epoch 4/5
1250/1250 [==============================] - 1084s 867ms/step - loss: 0.0079 - acc: 0.9980 - val_loss: 0.0234 - val_acc: 0.9935

Epoch 00004: val_loss did not improve
Epoch 5/5
1250/1250 [==============================] - 1084s 867ms/step - loss: 0.0065 - acc: 0.9983 - val_loss: 0.0178 - val_acc: 0.9954

Epoch 00005: val_loss did not improve


In [11]:
predict_on_xception(12500, 299, 299, test_data_dir, model, "xception-best_weight_fine_tuning-3.h5", "pred-xception-fine_tuning-3.csv")

100%|██████████| 12500/12500 [00:43<00:00, 286.42it/s]


12500/12500 [==============================] - 129s 10ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:166: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 247713.45it/s]


最终，Fine tuning的得分为：0.03770。

## 保存效果最好的那个模型
从最终的得分看出，目前效果最好的模型是Fine-tuning-2。我们载入全部模型，并将基础模型部分的权重分开保存起来。

In [ ]:
base_model.save_weights("fine_tuned_xception.h5")